In [190]:
# Data Wrangling
import pandas as pd
import numpy as np

# Eventual Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Text Processing
import emoji
import re
import string

# NLP
import spacy

# Language Detection
from transformers import pipeline

# Spell Checking
import enchant
from autocorrect import Speller
from enchant.checker import SpellChecker

In [191]:
# Ready scraped data
df = pd.read_csv("./data/merged_output.csv")

In [192]:
df.loc[(df["offer_ref"].isin([119352308, 126556830, 97049775,]) 
        & df["entry_id"].isin([16919388, 16453304, 16368349,])),
        "review_text"]

Series([], Name: review_text, dtype: object)

In [193]:
# Delete reviews written in cyrillic with a mix of polish characters (Those cases are not handled by cyryllic fix later on)
df = df[~(df["offer_ref"].isin([119352308, 126556830, 97049775,]) 
          & df["entry_id"].isin([16919388, 16453304, 16368349,]))]

In [194]:
other_langs = pd.read_csv("./data/foreign_languages.csv")

In [195]:
df.loc[df["entry_id"].isin(other_langs["entry_id"]) 
       & df["offer_ref"].isin(other_langs["offer_ref"]), 
       "review_text"].head(5)

Series([], Name: review_text, dtype: object)

In [196]:
# Delete reviews written in other languages
df = df.loc[~(df["entry_id"].isin(other_langs["entry_id"]) 
              & df["offer_ref"].isin(other_langs["offer_ref"]))]

In [197]:
# "review_text" nan are to be deleted. Other can stay.
df = df.dropna(subset=["review_text"])

# Remove invalid entries
df = df.loc[df["entry_date"] != "entry_date"]

# Remove duplicate entries
df = df.drop_duplicates(["offer_ref", "entry_id", "review_text"]).drop_duplicates("review_text")

# fix data types
df["entry_date"] = pd.to_datetime(df["entry_date"])
df["purchase_date"] = pd.to_datetime(df["purchase_date"])
df["entry_id"] = df["entry_id"].astype(int)
df["offer_ref"] = df["offer_ref"].astype(int)
df["score"] = df["score"].astype(float)

# Get Sentiment Cases based on score
df["sentiment"] = df["score"].apply(lambda x: "Positive" if x >= 4 else "Negative" if x <= 2 else "Neutral")

In [198]:
# Get only text and sentiment
to_clean = df[["review_text", "sentiment"]].copy()

In [207]:
# 1927 reviews containing special unicode characters
to_clean[to_clean["review_text"].str.contains("[\t\r\n\v\f\ufeff]")].shape

,review_text,sentiment,has_emoji,all_emoji


In [199]:
# Get rid of newlines and other whitespace chars
to_clean["review_text"] = to_clean["review_text"].replace("[\t\r\n\v\f\ufeff]", " ", regex=True)

In [200]:
# Remove whitespaces created by previous step
to_clean["review_text"] = to_clean["review_text"].replace(" +", " ", regex=True)
to_clean[to_clean["review_text"].str.contains("  ")]

,review_text,sentiment


In [201]:
# Remove cyrillic characters. Sometimes there are reviews half written in cyrillic and half in polish, so I will keep polish characters.
cyrylic_regex = f"[ {string.punctuation}0-9]*[\u0400-\u04FF]+[ {string.punctuation}0-9]*[\u0400-\u04FF]+[ {string.punctuation}0-9]*"
to_clean["review_text"] = to_clean["review_text"].transform(lambda x: re.sub(cyrylic_regex, "", x)).replace("", np.nan)
to_clean = to_clean.dropna(subset=["review_text"])

In [202]:
# model_ckpt = "papluca/xlm-roberta-base-language-detection"
# pipe = pipeline("text-classification", model=model_ckpt)
# language_classification = pipe(to_clean["review_text"].transform(lambda x: x[:100]).to_list(), top_k=1, truncation=True)
# to_clean = pd.concat([to_clean.reset_index(), pd.DataFrame(np.array(language_classification).reshape(-1).tolist())], axis=1).set_index("index")

In [203]:
# # Lot of Wrong predictions. I will fix them manually
# to_clean[(to_clean["label"] != "pl")].iloc[2400:]

In [204]:
# Mark all emojis in data. All_emoji will be excluded entirely, and has_emoji will remove only emojis.
to_clean["has_emoji"] = to_clean["review_text"].transform(lambda x: np.any([emoji.is_emoji(c) for c in x]))
to_clean["all_emoji"] = to_clean["review_text"].transform(lambda x: np.all([emoji.is_emoji(c) for c in x]))

In [205]:
# 460 Reviews containing emojis
print(len(to_clean[to_clean["has_emoji"]]))
# 42 reviews containing only emojis
print(len(to_clean[to_clean["all_emoji"]]))

676
55


In [206]:
# You can sort of guess the sentiment of the review based on the emojis. But it's not consistent.
to_clean[to_clean["all_emoji"]].head(10)

,review_text,sentiment,has_emoji,all_emoji
0,😑😑,Negative,True,True
1091,😐,Negative,True,True
9421,👍,Neutral,True,True
10053,👍👍👍👍👍,Positive,True,True
10958,🤗,Positive,True,True
11179,🥳🎉,Positive,True,True
13728,👍🏻👍🏻,Negative,True,True
14711,👍👍,Neutral,True,True
24204,🙂,Positive,True,True
25324,😫,Negative,True,True


In [222]:
# Remove all reviews containing single repeating characters that are non-emoji
to_clean = to_clean.loc[~(to_clean["review_text"].transform(lambda x: len(set(x)) == 1) & ~to_clean["all_emoji"])]

In [236]:
# remove reviews with only 3 unique characters that are not meaninful. Decided by hand.
# Meaningfull list contains different versions of emoticons and ok and kk and bdb and x... and 5/5 etc.
to_clean = to_clean[
    ~(to_clean["review_text"].transform(lambda x: len(set(x)) < 3) 
    & ~to_clean["has_emoji"] 
    & ~to_clean["review_text"].str.lower().str.contains("(ok|[0-9]|:[\)\(]|;[\)\()\/]|\*|bdb|kk|x)"))
]

/tmp/ipykernel_6282/585859017.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  & ~to_clean["review_text"].str.lower().str.contains("(ok|[0-9]|:[\)\(]|;[\)\()\/]|\*|bdb|kk|x)"))


In [11]:
# Mark Ok's in data.
to_clean["has_ok"] = to_clean["review_text"].str.lower().str.contains("\Wok\W")
to_clean[to_clean["has_ok"]] = to_clean[to_clean["has_ok"]].replace(r"\bok\b", "Ok", regex=True)

In [12]:
to_clean[to_clean["has_ok"] & (to_clean["sentiment"] != "Positive")]["review_text"].iloc[2]

'Kupiłem za namową sąsiadów, którzy zachwycali się możliwością pieczenia 2 potraw jednocześnie. Dual Cooki nie są drogie, miałem Ok. 2000 zł na piekarnik (poprzedni zepsuł się po 13 latach, ale wiem, że takich urządzeń już nie ma...). Zdecydowałem się na ten model choć już w sklepie pojawiły się pierwsze wątpliwości... drzwiczki piekarnika trzaskają jak w maluchu, obudowa drzwi jest w pełni plastikowa i nie znalazłem żadnego uszczelnienie tej przegrody rozdzielającej piekarnik na 2 części. Sprzedawca stwierdził, że są to bzdety nie mające wpływu na jakość pieczenia i szczerze mówiąc przekonał mnie. Piekę głównie mięsiwa, a żona ciasta - piekarnik działa praktycznie codziennie, używamy go także do podgrzewania (nie lubię mikrofali). Rozczarowanie przyszło już po pierwszym pieczeniu - piekarnik bardzo długo się nagrzewa - temperaturę 220 stopni osiągnął po 20 minutach, mimo, że termostat pokazywał taką temperaturę już po 8-10 minutach - własny termometr pokazał, że piekarnik przekłamuje.

In [13]:
# fix_ok_in_string assumes that ok. (circa) happens once per review. We can safely assume that.
to_clean[to_clean["review_text"].transform(lambda x: len(re.findall(r"\bok\b.? [0-9]+", x))) > 1]

,review_text,sentiment,has_emoji,all_emoji,has_ok


In [14]:
# Function discriminates between ok meaning good and ok meaning circa 
# which is a short form of około in polish language.
# Ok has it's own fix because it is very common in reviews and gives positive sentiment.
def fix_ok_in_string(text):
    circa_match = re.search(r"\bok\b.? [0-9]+", text.lower())
    if circa_match is not None:
        circa_match = circa_match.span()
        return re.sub(r"\b(ok|Ok|OK|oK)\b", "Ok", text[:circa_match[0]]) + text[circa_match[0]:circa_match[1]] + re.sub(r"\b(ok|Ok|OK|oK)\b", "Ok", text[circa_match[1]:])
    else:
        return re.sub(r"\b(ok|Ok|OK|oK)\b", "Ok", text)

In [15]:
fix_ok_in_string("ok. lalala ok. 100kg ok!")

'Ok. lalala ok. 100kg Ok!'

In [16]:
# Fix all ok's in data.
to_clean["review_text"] = to_clean["review_text"].transform(fix_ok_in_string)

In [17]:
# Remove all numbers from data.
to_clean["review_text"] = to_clean["review_text"].transform(lambda x: re.sub("[0-9]+", "", x))

In [15]:
# Largest default polish language model
nlp = spacy.load("pl_core_news_lg")

# Autocorrect speller
spell = Speller("pl", only_replacements=True)

# Enchant spellcheckers
chkr = SpellChecker("pl_Pl") 
d_typo = enchant.Dict("pl_PL")

In [16]:
to_clean["review_text"] = to_clean["review_text"].transform(lambda x: nlp(x))

In [62]:
to_clean["Typo Data"] = (
    to_clean["review_text"].transform(lambda x: 
        [[token.text, 
          d_typo.suggest(token.text), 
          i, 
          str(x).find(token.text), 
          len(token.text)] 
        for i, token in enumerate(x) 
            if not token.is_punct 
               and (not d_typo.check(token.text)) 
               and (emoji.is_emoji(token.text) == False)])
)

In [64]:
to_clean.to_pickle("./data/typos_annotated.pkl")

In [ ]:
# Will run 3 mins

# Get subset of data.
checker_test = to_clean["review_text"].iloc[:2000].transform(lambda x: nlp(x))

# 42% of first 2000 reviews contain typos
# pyenchany seems good at detecting typos however it seems bad at fixing typos So we'll have to be careful with it.
checker_test[checker_test.transform(lambda x: all([d_typo.check(token.text.lower()) for token in x if not token.is_punct])) == False]

In [20]:
# 5 Mins. tokenize data
to_clean["review_text"] = to_clean["review_text"].transform(nlp)

In [21]:
## Takes 44 mins.
## Fix all typos. Write 2 csv files with and without type fixing. Used for comparison in another script.
to_clean["review_text"].transform(lambda x: [d_typo.suggest(token.text)[0] if (not token.is_punct) and (not d_typo.check(token.text)) and (chkr.suggest(token.text))  else token for token in x]).to_csv("Testing Typo Checking.csv")
to_clean["review_text"].to_csv("Comparison to a Typo fix.csv")

In [22]:
# I've forgotten about sentiment.
to_clean["sentiment"].to_csv("sentiment provision.csv")